# Semantics and Pragmatics, KIK-LG103

## Lab session 2, Part 1

---

### Section 1.1. Review

In this section you can quickly review what we did in the last session. The section consists of example code that you can run (by pressing Ctrl-Enter inside the cell you want to run). You don't have to modify any code in this section. Feel free to continue directly to Section 1.2 below if you are comfortable with strings, lists and sets. 


#### Strings and lists

- Splitting strings to lists
- Iteration through lists
- Calculating the length of a list

First, let's print a sentence as a string:

In [ ]:
# Assign the sentence to a variable
sentence_as_string = "this is just another string"
print(sentence_as_string)

Next, let's split the sentence into a list. Each word in the sentence becomes an element in the list:

In [ ]:
# Split string on whitespace, the result is a list
sentence_as_list = sentence_as_string.split()
print(sentence_as_list)

To know how many elements there are in a list, we use the _len_ function. We typically talk about the length of a list. The length is the number of elements in the list.

In [ ]:
# len-function returns the length of the list
amt_tokens = len(sentence_as_list)
print("The sentence has", amt_tokens, "tokens.")

We can also determine the length of a string. How does this differ from the length of a list?

In [ ]:
# len-function also returns something for the string. What does it return?
len_of_string = len(sentence_as_string)
print("The length of the sentence as a string is", len_of_string)

Using a _for_ loop we can iterate through a list, one element at a time. Here we go through our sentence as a list and print every word in the sentence, one by one:

In [ ]:
# Iterate through the list and print every element (token)
for token in sentence_as_list:
    print(token)

We can also iterate through a string. How does this differ from iterating through a list?

In [ ]:
# Iterate through the string and print every element in the string. What are these elements, actually?
for c in sentence_as_string:
    print(c)

#### Sets

- Set as a collection of unique elements

Let's look at the difference between lists and sets:

In [ ]:
sentence = "the cat saw the dog"
tokens_in_sentence = sentence.split()

# set-function takes in a list and returns a set
types_in_sentence = set(tokens_in_sentence)
print("The set-function results in a:", type(types_in_sentence), "data structure.")

Pay attention to how the outputs of the prints differ:

In [ ]:
print("Tokens in sentence:", tokens_in_sentence)
print("Types in sentence:", types_in_sentence)

Len also works with sets:

In [ ]:
amt_types = len(types_in_sentence)
print("The string has", amt_types, "types.")

#### Functions & Flow control

- function: input $\rightarrow$ process $\rightarrow$ output
- if-else statement
- booleans

Make sure that you understand what goes on in the fun function below: 

In [ ]:
def fun(word, list_of_words):
    # the 'x in list' construct returns True if x
    # is in list and False otherwise
    if word in list_of_words:
        print("'" + word + "' is in the list.")
    else:
        print("'" + word + "' is not in the list.")
        
list1 = ["bananas", "excalibur", "fun"]

fun("bananas", list1)
fun("pineapple", list1)

---
## Section 1.2: Objects and methods
---
In this section we offer a brief introduction to two new programming concepts, **objects** and **methods**. Objects and methods are fundamental in Python programming (or at least the style of programming we learn in this course), and in fact you have already worked with both of them. Let's start with a familiar example:

In [ ]:
sentence = "This is a sentence" # This assignment creates a string object...

tokens = sentence.split()       # ...and here we use a split-method
                                # the object offers
print(sentence)
print(tokens)

Whenever we assign a string to some variable, we actually create a *string object* with the specified character sequence as its contents. So what's the deal with the variable now containing an object instead of just a plain old string like we have been saying all the time? An object is simply an instantiation of some pre-defined arbitrary data structure. The definition of the structure is called a class. We won't have to define our own classes, but keep the term in mind. The class is in other words a template for creating objects of the type, and it specifies what is common between all its instantiations, leaving the details to the individual objects. For example, the definition of the string class includes something about the contents being a sequence of characters, but the specific sequence is decided only when we create a string object. 

As we saw in the code cell above, the `split()` is actually a method of the string. A method is a special kind of function tied to some specific class, and it is called with the dot-notation (`object.method()`). The distinction between methods and functions is subtle. You might think of functions as feeding your data to some process or machine and later gathering the result of the process for your use. In the case of methods, you tell the entity containing the data what you want of it, and the entity gives you the result. There is no reason in principle why you could not implement your own `split`-function that would take as an argument a string and return a list, and use that instead of the `split`-method.

Fortunately, we have a great example of a task we can implement both as a function and a method. In the last lab session we implemented a function `ends_with(string1, string2)` that takes two arguments and returns a `boolean` indicating whether `string1` ends with `string2`. You can find the definition below. 

Implementing the `ends_with` function was a good illustration, but in fact unnecessary work. The Python string class already supplies a method `endswith()` that takes one string argument and returns a boolean indicating whether the string on which we are calling the method ends with the argument string.

In [ ]:
def ends_with(string1, string2):
    string_end = string1[-len(string2):]
    return string_end == string2

# Here we use the function we defined above
print(ends_with("string", "ing"))

# Here we use the 'endswith' method Python strings supply
s = "string"
print(s.endswith("ing"))

In the rest of this lab we will use more complex classes and methods supplied by WordNet. Keep these ideas in mind as you work with the code.